In [1]:
from nltk.corpus import gutenberg

In [3]:
import nltk;
nltk.corpus.gutenberg.fileids()


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [11]:
a= nltk.corpus.gutenberg.fileids()

In [29]:
print(gutenberg.raw('austen-emma.txt')[0:1000])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.  Between _them_ it was more the intimacy
of sisters.  Even before Miss Taylor had ceased to hold the nominal
office of governess, the mildness o

In [12]:
from nltk.corpus import gutenberg   # Docs from project gutenberg.org
import re
files_en = gutenberg.fileids()      # Get file ids
doc_en = gutenberg.open('edgeworth-parents.txt').read()

tokens_en = re.sub('[^A-Za-z]', ' ', doc_en)
#nltk.download('gutenberg')
tokens_en_lower = tokens_en.lower()

from nltk.tokenize import word_tokenize
tokens_en_lower = word_tokenize(tokens_en_lower)
from nltk.corpus import stopwords 
filtered_words = [word for word in tokens_en_lower if word not in stopwords.words('english')]

en = nltk.Text(filtered_words)

print(len(en.tokens))       # returns number of tokens (document length)
print(len(set(en.tokens)))  # returns number of unique tokens
en.vocab() 

#en.plot(50)
print(len(doc_en))
print(tokens_en.count('Emma'))
print(en.count('Emma'))

78118
8169
935158
0
0


In [13]:
len(filtered_words)

78118

In [17]:
def sampling_6(data, n=10):
    data=copy.deepcopy(data);
    sample=[];
    for i in range(n):
        idx = random.randint(0,len(data)-1);
        a =sample.append(data.pop(idx));
    return sample

In [18]:
import random
import copy
b=[]
for i in range(200): 
    a=sampling_6(filtered_words, n=150)
    b.append(a)
    

In [56]:
import pandas as pd
df = pd.DataFrame(b)
f= pd.DataFrame([], columns=["Sample"])

In [57]:
f

,Sample


In [58]:
f=df.apply(" ".join, axis=1)

In [54]:
f

0      fast determined next see shall wish running li...
1      wonder whether summer ever know laughed slippe...
2      though know pounds turn took gardener mind sea...
3      dared see lawrence way cups hold question come...
4      recollect cook recollect sees washing hands fi...
5      returning day would shillings truth go round j...
6      could could taken days servants hands console ...
7      waistcoat crowds things assure mrs delicate jo...
8      threatened scoriae waiter fast archers cap cro...
9      curtsied party nothing children order eyes com...
10     make asked cried cried money tarlton said prov...
11     love procure one shall nine room trifles frien...
12     warm surrounded several hand boat firm hand mu...
13     arms second pleasing twelve contempt believe g...
14     whole know upon generous whispered prize took ...
15     summoned transactions pray went companions pre...
16     wheel man miss playing describe ten driving pi...
17     children friend thus wic

In [55]:
f[0]

'fast determined next see shall wish running little rapture father tune uniform delicate first stay looked protect ever said door came young butcher dead faced paper sugar lengthened oh yards foot every de immediately parcel lost greco sometimes newington plainly last grey shall right let beg mats bogberry going john see nothing winning hatful whether market window employer constantly susan pies rendered ear triumphant everybody end guard everyone chambermaid sarve come father improve col yard hard likely moment song forms stuff say indeed ornament give therefore tell said ask sir means patients boy louisa large corner replied entrance blessing bit gardener wanted horse procured shall eat cautiously know bursal mad look hypocrite whole old sick poor franklin several honest foot better marianne food boy melancholy man employment known supper disorder greater glass frederick spin come best never boy trimmed accident back piece ran find litigious married figure thought hat flashed'

In [30]:
len(f[0])

991

In [31]:
len(f[15])

926

In [37]:
f[1]

'wonder whether summer ever know laughed slipped threw mr said show cried paradise poor ridgeway desirous dialogue quarter observed pshaw dressed bill arm quiet tight nephews refuges without money lightfoot said sure almonds advantage hear shoe archer saw perhaps made peddler help night said usually hold things brother passions head come would abbey dead also heard always matter pardon piedro seems either never went shilling sir afraid last neighbour things thanks message put caught friends quite nets substantially ready make good shillings jingle used ignorance house travelling head came species said mind unlock servants way dexterously else bridewell know love eggs talb twopence padrona means displeased wood whilst meant ha help lady saw maurice take third see china well upper provisions would mistake rose loveit fancy bracelet loveit mr miss gardener yet see guinea thinks look give loveit recollecting wish yesterday arrival completely wrong far cried former piedro see father'

In [36]:
for i in range(15):
    print(f[i])


fast determined next see shall wish running little rapture father tune uniform delicate first stay looked protect ever said door came young butcher dead faced paper sugar lengthened oh yards foot every de immediately parcel lost greco sometimes newington plainly last grey shall right let beg mats bogberry going john see nothing winning hatful whether market window employer constantly susan pies rendered ear triumphant everybody end guard everyone chambermaid sarve come father improve col yard hard likely moment song forms stuff say indeed ornament give therefore tell said ask sir means patients boy louisa large corner replied entrance blessing bit gardener wanted horse procured shall eat cautiously know bursal mad look hypocrite whole old sick poor franklin several honest foot better marianne food boy melancholy man employment known supper disorder greater glass frederick spin come best never boy trimmed accident back piece ran find litigious married figure thought hat flashed
wonder w

In [111]:
f = pd.DataFrame(f, columns=['Sample'])


In [112]:
f

,Sample
0,fast determined next see shall wish running li...
1,wonder whether summer ever know laughed slippe...
2,though know pounds turn took gardener mind sea...
3,dared see lawrence way cups hold question come...
4,recollect cook recollect sees washing hands fi...
5,returning day would shillings truth go round j...
6,could could taken days servants hands console ...
7,waistcoat crowds things assure mrs delicate jo...
8,threatened scoriae waiter fast archers cap cro...
9,curtsied party nothing children order eyes com...


In [113]:
c = pd.DataFrame({'author':[],'Genre':[]})

In [114]:
c

,author,Genre


In [115]:
f 

,Sample
0,fast determined next see shall wish running li...
1,wonder whether summer ever know laughed slippe...
2,though know pounds turn took gardener mind sea...
3,dared see lawrence way cups hold question come...
4,recollect cook recollect sees washing hands fi...
5,returning day would shillings truth go round j...
6,could could taken days servants hands console ...
7,waistcoat crowds things assure mrs delicate jo...
8,threatened scoriae waiter fast archers cap cro...
9,curtsied party nothing children order eyes com...


In [116]:
frames =[f,c]

In [117]:
result = pd.concat(frames)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [118]:
result = result[['Sample','author','']]

,Genre,Sample,author
0,NaN,fast determined next see shall wish running li...,NaN
1,NaN,wonder whether summer ever know laughed slippe...,NaN
2,NaN,though know pounds turn took gardener mind sea...,NaN
3,NaN,dared see lawrence way cups hold question come...,NaN
4,NaN,recollect cook recollect sees washing hands fi...,NaN
5,NaN,returning day would shillings truth go round j...,NaN
6,NaN,could could taken days servants hands console ...,NaN
7,NaN,waistcoat crowds things assure mrs delicate jo...,NaN
8,NaN,threatened scoriae waiter fast archers cap cro...,NaN
9,NaN,curtsied party nothing children order eyes com...,NaN


In [109]:
for i in range(200):
    result['Genre'] ="Fiction"
    result['author'] ="Edgeworth"

In [110]:
result

,Genre,Sample,author
0,Fiction,fast determined next see shall wish running li...,Edgeworth
1,Fiction,wonder whether summer ever know laughed slippe...,Edgeworth
2,Fiction,though know pounds turn took gardener mind sea...,Edgeworth
3,Fiction,dared see lawrence way cups hold question come...,Edgeworth
4,Fiction,recollect cook recollect sees washing hands fi...,Edgeworth
5,Fiction,returning day would shillings truth go round j...,Edgeworth
6,Fiction,could could taken days servants hands console ...,Edgeworth
7,Fiction,waistcoat crowds things assure mrs delicate jo...,Edgeworth
8,Fiction,threatened scoriae waiter fast archers cap cro...,Edgeworth
9,Fiction,curtsied party nothing children order eyes com...,Edgeworth
